In [1]:
import matplotlib.pyplot as plt
import geopandas as gpd
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd 
import folium
import seaborn as sns

In [ ]:
moskva = pd.read_csv('clean_data_MSK.csv', sep=':')
moskva.head()

In [6]:
moskva_new = moskva[(moskva['Долгота'] < 37.9545100) & (moskva['Долгота'] > 37.1813900) & (moskva['Широта'] <55.9825000) & (moskva['Широта'] > 55.1339600)].copy()

In [ ]:
categor_df = moskva_new[['Тип аварии', 'Количество погибших',	'Количество пострадавших', 'Количество участников']].groupby('Тип аварии').sum()
categor_df["P_death"] = round(categor_df['Количество погибших'] / categor_df['Количество участников'], 2)
categor_df

In [ ]:
map_ = folium.Map([55.75215, 37.61819], zoom_start=10)
for id, row in moskva_new.iterrows():
    folium.Circle([row['Широта'], row['Долгота']],
                  radius=10).add_to(map_)
map_

In [12]:
from sklearn.cluster import DBSCAN

"""clast = DBSCAN(eps=0.001, 
            min_samples=15, 
            metric='euclidean')
clast.fit(moskva_new[['Lat', 'Long']].values)
"""

distance = 0.1 # in kilometers
clast = DBSCAN(eps=distance/6371., min_samples=15, metric='haversine')
clast.fit(np.radians(moskva_new[['Широта', 'Долгота']].values))


DBSCAN(eps=1.5696123057604773e-05, metric='haversine', min_samples=15)

In [ ]:
DBSCAN_clustered_map(moskva_new, clast, label_colour='purple')

In [102]:
from matplotlib.colors import rgb2hex
from matplotlib.cm import rainbow


def DBSCAN_clustered_map(data, cluster_model, label_colour='purple'):
    '''
    Create map with coloured clusters
    param data: pd.DataFrame - public transport stops data
    param labels: (n_samples, ) - cluster labels for each stop
    return: folium.Map - map with coloured clusters
    '''
    labels = cluster_model.labels_
    df = data[['Lat', 'Long', 'severity']].copy()
    df.insert(2, "labels", labels + 1)

    Moscow_Map = folium.Map([55.75215, 37.61819], zoom_start=11)
    for id, row in df.iterrows():
        idx = row.labels - 1
        if idx >= 0:
           if row.severity=='С погибшими':
              folium.Circle([row.Lat, row.Long],
                    radius=40, color='red').add_to(Moscow_Map)
           elif row.severity=='Тяжёлый':
              folium.Circle([row.Lat, row.Long],
                    radius=26, color=label_colour).add_to(Moscow_Map)
           else:
              folium.Circle([row.Lat, row.Long],
                    radius=5, color='lightgreen').add_to(Moscow_Map)
    
    
    return Moscow_Map

In [21]:
from sklearn.model_selection import train_test_split